<a href="https://colab.research.google.com/github/PremGorecki/NeuralNetwork/blob/main/05_regression/01_housing_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import biblotek

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
tf.__version__

'2.8.2'

# 2. Załadowanie danych i wstępna eksploracja

In [ ]:
raw_dataset = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ann-course/housing.csv')
raw_dataset.info()

In [ ]:
# dobra praktyka to skopiować DataSet
dataset = raw_dataset.copy()
dataset.head()

In [ ]:
#sprawdzamy ile pustych danych
dataset.isnull().sum() / len(dataset)

In [ ]:
#danych pustych poniżej 1% to można usunąć

dataset.dropna(inplace=True)

dataset.isnull().sum() / len(dataset)

In [ ]:
#zwróci nam pewne statystyki ale tylko dla zmiennych liczbowych

dataset.describe()

In [ ]:
#możemy też mieć statystyki dla zmiennych nieliczbowych
dataset.describe(include=['object'])

In [9]:
dataset.ocean_proximity.value_counts()

<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2628
NEAR BAY      2270
ISLAND           5
Name: ocean_proximity, dtype: int64

In [ ]:
#jak wygląda rozkład zmiennej którą chcemy przywidywać
px.histogram(dataset, x='median_house_value')

In [ ]:
#widzimy że 500001 jest często
dataset.median_house_value.value_counts()

In [ ]:
#usuwamy 5000001 
index_to_drop = dataset[dataset.median_house_value == 500001].index
dataset = dataset.drop(index=index_to_drop)
px.histogram(dataset, x='median_house_value')

In [ ]:
#teraz ostatnią kolumnę gdzie tym był object rozbijamy to na 5 kolumn z zerami i jedynkami
dataset_dummies = pd.get_dummies(dataset, drop_first=True)
dataset_dummies.head()

# 3. Podział na zbiór treningowy oraz testowy

In [ ]:
train_dataset = dataset_dummies.sample(frac=0.8, random_state=0)
test_dataset = dataset_dummies.drop(train_dataset.index)

print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset length: {len(test_dataset)}')

In [ ]:
train_dataset.head()

In [ ]:
test_dataset.head()

In [ ]:
# eksplorujemy dane bardziej
px.scatter_matrix(train_dataset, dimensions=['median_house_value', 'housing_median_age', 'median_income', 'total_rooms'], color='median_house_value', height=700)

In [ ]:
#statystyki ale bez cen bo to będziemy przewidywać
train_stats = train_dataset.describe()
train_stats.pop('median_house_value')
train_stats = train_stats.transpose()
train_stats

In [ ]:
#usuwamy ceny z danych
train_labels = train_dataset.pop('median_house_value')
test_labels = test_dataset.pop('median_house_value')

# 4. Standaryzacja danych

In [21]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [22]:
#uwaga funkcja działa na danych treningowych a potem uruchamiamy ją też na danych testowych
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
normed_train_data.isnull().sum()

In [25]:
normed_test_data = normed_test_data.values
normed_train_data = normed_train_data.values

# 5. Budowa modelu

In [26]:
def build_model():
    model = Sequential()
    model.add(Dense(1024, kernel_regularizer='l2', activation='relu', input_shape=[len(train_dataset.keys())]))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model

In [ ]:
model = build_model()
model.summary()

# 6. Trenowanie sieci

In [28]:
history = model.fit(normed_train_data, train_labels.values, epochs=150, validation_split=0.2, verbose=1, batch_size=32)

Epoch 1/150
390/390 [==============================] - 4s 9ms/step - loss: 17058109440.0000 - mae: 95162.5391 - mse: 17058109440.0000 - val_loss: 4729602560.0000 - val_mae: 49529.7188 - val_mse: 4729602560.0000
Epoch 2/150
390/390 [==============================] - 3s 8ms/step - loss: 4258263552.0000 - mae: 46686.5586 - mse: 4258263552.0000 - val_loss: 3820925440.0000 - val_mae: 44749.6953 - val_mse: 3820925440.0000
Epoch 3/150
390/390 [==============================] - 3s 8ms/step - loss: 3783534336.0000 - mae: 44308.0938 - mse: 3783534336.0000 - val_loss: 3599489280.0000 - val_mae: 44060.1836 - val_mse: 3599489280.0000
Epoch 4/150
390/390 [==============================] - 3s 8ms/step - loss: 3628985856.0000 - mae: 43266.1328 - mse: 3628985856.0000 - val_loss: 3488358912.0000 - val_mae: 42639.6445 - val_mse: 3488358912.0000
Epoch 5/150
390/390 [==============================] - 3s 8ms/step - loss: 3547198976.0000 - mae: 42739.5781 - mse: 3547198976.0000 - val_loss: 3422228992.0000 - 

In [ ]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist['rmse'] = np.sqrt(hist['mse'])
    hist['val_rmse'] = np.sqrt(hist['val_mse'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['mae'], name='mae', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_mae'], name='val_mae', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MAE vs. VAL_MAE', xaxis_title='Epoki', yaxis_title='Mean Absolute Error', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['rmse'], name='rmse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_rmse'], name='val_rmse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='RMSE vs. VAL_RMSE', xaxis_title='Epoki', yaxis_title='Root Mean Squared Error', yaxis_type='log')
    fig.show()

plot_hist(history)

In [ ]:
for name, value in zip(model.metrics_names, model.evaluate(normed_test_data, test_labels.values)):
    print(f'{name:8}{value:.4f}')

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

In [ ]:
pred = pd.DataFrame(test_labels)
pred['predictions'] = test_predictions
pred.head()

In [ ]:
fig = px.scatter(pred, 'median_house_value', 'predictions')
fig.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines'))
fig.show()

In [ ]:
pred['error'] = pred['median_house_value'] - pred['predictions']
pred.head()

In [ ]:
px.histogram(pred, 'error', marginal='rug', width=1000)